# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests
# Configurar para exibir mais linhas e colunas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Configurar para mostrar todo o conteúdo das colunas
pd.set_option('display.max_colwidth', None)

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'

# Fazendo a requisição HTTP GET
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    # Convertendo a resposta JSON para um dicionário Python
    data = response.json()
    
    # Convertendo o dicionário para um DataFrame do Pandas
    # A chave 'items' contém a lista de registros
    df = pd.json_normalize(data['items'])
    
    # Exibindo as primeiras linhas do DataFrame para verificar os dados carregados
    print(df.head(10))
else:
    print(f'Falha na requisição. Status code: {response.status_code}')

   id_pleito tipo_interessado     interessado  cod_ibge  uf num_pvl  \
0       8528           Estado  Rio de Janeiro        33  RJ    None   
1       7057           Estado  Rio de Janeiro        33  RJ    None   
2       7084           Estado  Rio de Janeiro        33  RJ    None   
3      13624           Estado  Rio de Janeiro        33  RJ    None   
4       7085           Estado  Rio de Janeiro        33  RJ    None   
5       5516           Estado  Rio de Janeiro        33  RJ    None   
6       7080           Estado  Rio de Janeiro        33  RJ    None   
7       8556           Estado  Rio de Janeiro        33  RJ    None   
8       9750           Estado  Rio de Janeiro        33  RJ    None   
9      10002           Estado  Rio de Janeiro        33  RJ    None   

                                                  status  \
0                                               Deferido   
1  Encaminhado à PGFN com manifestação técnica favorável   
2  Encaminhado à PGFN com manifestação

In [3]:
# 2) Seu código aqui
df['status'].value_counts()

status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: count, dtype: int64

In [62]:
# 3) Seu código aqui 
# df['ano_status'] = df['data_status'].dt.year
print(df['data_status'].dtype)
df['ano_status'] = df['data_status'].astype(str).str[-4:]
df['ano_status'].head(10)

object


0    2009
1    2014
2    2010
3    2002
4    2012
5    2011
6    2012
7    2011
8    2006
9    2009
Name: ano_status, dtype: object

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [30]:
#1) Seu código aqui
def baixar_dataframe_uf(uf):
    url = f'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado=Estado'
    response = requests.get(url)# Verificando se a requisição foi bem-sucedida
    
    if response.status_code == 200:
        # Convertendo a resposta JSON para um dicionário Python
        data = response.json()
        
        # Convertendo o dicionário para um DataFrame do Pandas
        # A chave 'items' contém a lista de registros
        df = pd.json_normalize(data['items'])
        
        # Exibindo as primeiras linhas do DataFrame para verificar os dados carregados
        df.head(10)
        return df
    else:
        print(f'Falha na requisição. Status code: {response.status_code}')


In [36]:
# 2) Seu código aqui
df_mg = baixar_dataframe_uf('MG')
df_mg['status'].value_counts()

# Arquivado por decurso de prazo = 1

status
Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Em retificação pelo interessado                           1
Indeferido                                                1
Arquivado por decurso de prazo                            1
Name: count, dtype: int64

In [59]:
# 3) Seu código aqui
df_ba = baixar_dataframe_uf('BA')

df_ba_deferido = df_ba[df_ba['status'] == 'Deferido']
contagem_interessado  = df_ba_deferido['interessado'].value_counts()

print(contagem_interessado)

# Não encontrei o campo município :( No entanto usei a coluna interessado como exemplo de como usaria o value_counts para contar quantos municipios
# existe entre os deferidos


interessado
Bahia    50
Name: count, dtype: int64


In [58]:
# 4) Seu código aqui
df_ba.to_csv('dados_ba.csv', index=False)